# ResNet CIFAR-10 using Keras

This notebook shows how to use [Keras](https://keras.io) with TensorFlow.

The model used for this notebook is a RestNet model, trained with the CIFAR-10 dataset. See the following papers for more background:

[Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Dec 2015.

[Identity Mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf) by Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun, Jul 2016.

## Setup

First, we need to create a SageMaker session and define the IAM role we will use to access SageMaker.

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### Download the CIFAR-10 dataset
Next we need a dataset to use for training and testing. Downloading the data will take around 5 minutes.

In [ ]:
import utils

utils.cifar10_download()

### Upload the data to an S3 bucket

In [ ]:
inputs = sagemaker_session.upload_data(path='/tmp/cifar10_data', key_prefix='data/cifar10')

`sagemaker_session.upload_data` will upload the CIFAR-10 dataset from your machine to an S3 bucket named `sagemaker-{region}-{your AWS account number}`. if you don't have this bucket yet, ``sagemaker_session`` will create it for you.

### Complete source code
- [source_dir/resnet_model.py](source_dir/resnet_model.py): ResNet model
- [source_dir/resnet_cifar_10.py](source_dir/resnet_cifar_10.py): main script used for training and hosting

This is where Keras comes into play. To use a Keras model, you need to implement the function ``keras_model_fn()`` in the training and hosting script. The Keras library is completely contained within the TensorFlow library, so there is no need to pip install a separate library to use Keras.

Here is the content of our ``keras_model_fn()`` for this example:

```python
import tensorflow as tf

def keras_model_fn(hyperparameters):
    inputs = tf.keras.layers.Input(shape=(HEIGHT, WIDTH, DEPTH))

    x = tf.keras.layers.Dense(NUM_CLASSES * 2, activation='relu')(inputs)
    outputs = tf.keras.layers.Dense(NUM_CLASSES)(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
```

## Create a training job using the sagemaker.TensorFlow estimator

The `fit()` method will create a training job named `tensorboard-example-{unique identifier}` on two ml.c4.xlarge instances.

In [ ]:
from sagemaker.tensorflow import TensorFlow

source_dir = os.path.join(os.getcwd(), 'source_dir')
estimator = TensorFlow(entry_point='resnet_cifar_10.py',
                       source_dir=source_dir,
                       role=role,
                       training_steps=1000, evaluation_steps=100,
                       train_instance_count=1, train_instance_type='ml.c4.xlarge',
                       base_job_name='keras-example')

estimator.fit(inputs)

## Deploy the trained model to prepare for predictions

The `deploy()` method creates an endpoint which serves prediction requests in real-time.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

# Cleaning up
To avoid incurring charges to your AWS account for the resources used in this tutorial you need to delete the **SageMaker Endpoint:**

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

### Note
Using Keras will not work for distributed training: https://github.com/tensorflow/tensorflow/issues/14504